# Day 13: 幾何学変換の応用

## Learning Objectives
- 拡大・縮小を実装する
- 回転・平行移動を理解する

## 拡大・縮小

In [ ]:
def resize_nearest(image, scale_x, scale_y):
    """最近傍補間でリサイズ
    
    Args:
        scale_x: 横方向の拡大率
        scale_y: 縦方向の拡大率
    """
    h, w = len(image), len(image[0])
    new_h = int(h * scale_y)
    new_w = int(w * scale_x)
    
    output = [[0 for _ in range(new_w)] for _ in range(new_h)]
    
    for y in range(new_h):
        for x in range(new_w):
            # 元画像の対応位置を計算
            src_y = int(y / scale_y)
            src_x = int(x / scale_x)
            output[y][x] = image[src_y][src_x]
    
    return output

# テスト画像
small_img = [
    [100, 150, 200],
    [120, 170, 220],
    [140, 190, 240]
]

print("元の画像 (3x3):")
for row in small_img:
    print(row)

# 2倍に拡大
enlarged = resize_nearest(small_img, 2, 2)
print("\n2倍拡大 (6x6):")
for row in enlarged:
    print(row)

# 0.5倍に縮小
shrinked = resize_nearest(small_img, 0.5, 0.5)
print("\n0.5倍縮小:")
for row in shrinked:
    print(row)

## 双一次補間

In [ ]:
def resize_bilinear(image, scale_x, scale_y):
    """双一次補間でリサイズ
    
    最近傍補間より滑らかな結果
    """
    h, w = len(image), len(image[0])
    new_h = int(h * scale_y)
    new_w = int(w * scale_x)
    
    output = [[0 for _ in range(new_w)] for _ in range(new_h)]
    
    for y in range(new_h):
        for x in range(new_w):
            # 元画像の座標（小数）
            src_y = y / scale_y
            src_x = x / scale_x
            
            # 整数部分と小数部分
            y0, x0 = int(src_y), int(src_x)
            dy, dx = src_y - y0, src_x - x0
            
            # 境界チェック
            y1 = min(y0 + 1, h - 1)
            x1 = min(x0 + 1, w - 1)
            
            # 双一次補間
            val = (1-dx)*(1-dy)*image[y0][x0] + dx*(1-dy)*image[y0][x1] + (1-dx)*dy*image[y1][x0] + dx*dy*image[y1][x1]
            output[y][x] = int(val)
    
    return output

# テスト画像
test_img = [
    [100, 150, 200],
    [120, 170, 220],
    [140, 190, 240]
]

print("元の画像 (3x3):")
for row in test_img:
    print(row)

# 双一次補間で2倍に拡大
bilinear_enlarged = resize_bilinear(test_img, 2, 2)
print("\n双一次補間で2倍拡大 (6x6):")
for row in bilinear_enlarged:
    print(row)

## 回転変換

In [ ]:
import math

def rotate_nearest(image, angle):
    """最近傍補間で回転
    
    Args:
        angle: 回転角度（度）
    """
    
    # 角度をラジアンに変換
    rad = math.radians(angle)
    cos_a = math.cos(rad)
    sin_a = math.sin(rad)
    
    h, w = len(image), len(image[0])
    
    # 回転後の画像サイズを計算
    new_h = int(h * abs(cos_a) + w * abs(sin_a))
    new_w = int(w * abs(cos_a) + h * abs(sin_a))
    
    # 画像中心座標
    center_x, center_y = w / 2, h / 2
    new_center_x, new_center_y = new_w / 2, new_h / 2
    
    output = [[0 for _ in range(new_w)] for _ in range(new_h)]
    
    for y in range(new_h):
        for x in range(new_w):
            # 回転の逆変換（回転後の座標 → 回転前の座標）
            src_x = (x - new_center_x) * cos_a + (y - new_center_y) * sin_a + center_x
            src_y = -(x - new_center_x) * sin_a + (y - new_center_y) * cos_a + center_y
            
            # 座標が整数部分を取るように
            src_x_int, src_y_int = int(src_x), int(src_y)
            
            # 境界チェック
            if 0 <= src_y_int < h and 0 <= src_x_int < w:
                output[y][x] = image[src_y_int][src_x_int]
    
    return output

# テスト画像
img_4x4 = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
]

print("元の画像 (4x4):")
for row in img_4x4:
    print(row)

# 90度回転
rotated_90 = rotate_nearest(img_4x4, 90)
print("\n90度回転:")
for row in rotated_90:
    print(row)

# 45度回転
rotated_45 = rotate_nearest(img_4x4, 45)
print("\n45度回転:")
for row in rotated_45:
    print(row)

## 平行移動

In [ ]:
def translate(image, dx, dy):
    """平行移動
    
    Args:
        dx: 横方向の移動量（正なら右）
        dy: 縦方向の移動量（正なら下）
    """
    h, w = len(image), len(image[0])
    
    # 出力画像サイズ（移動先を収めるために余白を追加）
    new_h = h + abs(dy)
    new_w = w + abs(dx)
    
    output = [[0 for _ in range(new_w)] for _ in range(new_h)]
    
    for y in range(new_h):
        for x in range(new_w):
            # 元画像の対応位置を計算
            src_y = y - dy
            src_x = x - dx
            
            # 境界チェック
            if 0 <= src_y < h and 0 <= src_x < w:
                output[y][x] = image[src_y][src_x]
    
    return output

# テスト画像
img_3x3 = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

print("元の画像 (3x3):")
for row in img_3x3:
    print(row)

# 右に2、下に1移動
translated = translate(img_3x3, 2, 1)
print("\n右に2、下に1移動:")
for row in translated:
    print(row)

## 練習問題

### 練習問題1: 縦横比維持の拡大縮小
横方向に2倍、縦方向に1.5倍の拡大を行う関数を実装してください。

In [ ]:
# 解答例
def resize_aspect_ratio(image, scale):
    """縦横比維持での拡大縮小
    
    Args:
        scale: 拡大率（横方向は2倍、縦方向は1.5倍）
    """
    h, w = len(image), len(image[0])
    new_h = int(h * 1.5 * scale)
    new_w = int(w * 2 * scale)
    
    output = [[0 for _ in range(new_w)] for _ in range(new_h)]
    
    for y in range(new_h):
        for x in range(new_w):
            # 元画像の対応位置を計算
            src_y = y / (1.5 * scale)
            src_x = x / (2 * scale)
            src_y_int, src_x_int = int(src_y), int(src_x)
            
            if 0 <= src_y_int < h and 0 <= src_x_int < w:
                output[y][x] = image[src_y_int][src_x_int]
    
    return output

# テスト
test_img = [
    [1, 2],
    [3, 4],
    [5, 6]
]

print("元の画像 (3x2):")
for row in test_img:
    print(row)

scaled = resize_aspect_ratio(test_img, 1)
print("\n拡大後 (横2倍×縦1.5倍 = 4x4):")
for row in scaled:
    print(row)

### 練習問題2: 任意角度の回転
30度、60度、120度の回転を実装してください。

In [ ]:
# 解答例
def test_rotation_angles(image):
    """様々な角度での回転テスト"""
    angles = [30, 60, 120]
    
    for angle in angles:
        print(f"\n{angle}度回転:")
        rotated = rotate_nearest(image, angle)
        for row in rotated:
            print(row)

# テスト画像
simple_img = [
    [1, 0],
    [0, 0]
]

test_rotation_angles(simple_img)

## まとめ

- 拡大縮小には最近傍補間と双一次補間がある
- 回転には三角関数を使った座標変換が必要
- 平行移動は単純な座標シフトで実現できる
- 縦横比を維持して拡大縮小する場合は、比率を考慮する必要がある